In [1]:
from os.path import join

import numpy as np
import json

In [2]:
DISEASE_OUT_FILE_PATH = join("..", "..", "diseases-out.json")

In [3]:
with open(DISEASE_OUT_FILE_PATH, "r") as f:
    contents = json.load(f)

In [4]:
json.dumps(contents[5])

'{"type": "item", "id": "Q145790", "labels": {"de": {"language": "de", "value": "Iridozyklitis"}, "en": {"language": "en", "value": "iridocyclitis"}, "es": {"language": "es", "value": "iridociclitis"}, "fi": {"language": "fi", "value": "Iridosykliitti"}, "ja": {"language": "ja", "value": "\\u8679\\u5f69\\u6bdb\\u69d8\\u4f53\\u708e"}, "ro": {"language": "ro", "value": "Iridociclit\\u0103"}, "ru": {"language": "ru", "value": "\\u0418\\u0440\\u0438\\u0434\\u043e\\u0446\\u0438\\u043a\\u043b\\u0438\\u0442"}, "nb": {"language": "nb", "value": "regnbuehinnebetennelse"}, "fr": {"language": "fr", "value": "iridocyclite"}, "el": {"language": "el", "value": "\\u0399\\u03c1\\u03b9\\u03b4\\u03bf\\u03ba\\u03c5\\u03ba\\u03bb\\u03af\\u03c4\\u03b9\\u03b4\\u03b1"}, "eo": {"language": "eo", "value": "irisociklito"}, "ar": {"language": "ar", "value": "\\u0627\\u0644\\u062a\\u0647\\u0627\\u0628 \\u0627\\u0644\\u0642\\u0632\\u062d\\u064a\\u0629 \\u0648 \\u0627\\u0644\\u062c\\u0633\\u0645 \\u0627\\u0644\\u06

In [52]:
dataset = []
tracked_claims = {
    "P279": "subclass_of",
    "P2579": "study_by",
    "P1995": "health_speciality",
    "P780": "symptoms_and_signs"
}

def map_list(it, fn):
    return [i for i in map(fn, it) if i is not None]

def retrieve(object_, key, default):
    keys = key.split('/')
    for key in keys:
        object_ = object_.get(key, None)
        if object_ is None:
            return default
    return object_

for content in contents:
    label_en = retrieve(content, 'labels/en/value', None)
    if label_en is None:
        continue

    elem = {
        "id": retrieve(content, "id", None),
        "name": label_en,
        "description": retrieve(content, 'descriptions/en/value', None),
        "alternative": map_list(retrieve(content, 'aliases/en', []), lambda x: retrieve(x, 'value', None)),
    }

    for key in tracked_claims.values():
        elem[key] = []

    claims_dict = retrieve(content, "claims", {})
    for claim_id in claims_dict:
        if not claim_id in tracked_claims.keys():
            continue

        for claim in claims_dict[claim_id]:
            value = retrieve(claim, "mainsnak/datavalue/value/id", None)
            elem[tracked_claims[claim_id]].append(value)

    dataset.append(elem)

In [63]:
[x["name"] for x in dataset if "HIV" in x["name"]]

['HIV seropositivity', 'HIV-associated lipodystrophy']